# Fundamental Analysis

link - https://python.plainenglish.io/fundamental-stock-analysis-using-python-apis-9988afdd4d24

In [1]:
import json

# For DataFrame
import pandas as pd
import numpy as np

In [13]:
# Download data

from yfinance3 import YFinance3
import json

SYMBOLS = ['SBIN.NS','ITC.NS','VBL.NS','MRPL.NS','ONGC.NS','HSCL.NS','FCL.NS']


for symbol in SYMBOLS:
    data = YFinance3(symbol)
    file_name = f'datasets/{symbol}.json'
    # Use a context manager to open the file and write the JSON data to it
    with open(file_name, 'w') as file:
        json.dump(data.info, file)
    print('saved to {}'.format(file_name))

saved to datasets/SBIN.NS.json
saved to datasets/ITC.NS.json
saved to datasets/VBL.NS.json
saved to datasets/MRPL.NS.json
saved to datasets/ONGC.NS.json
saved to datasets/HSCL.NS.json
saved to datasets/FCL.NS.json


In [14]:
# List of stock symbols we need to run fundamental analysis on - any symbol added here must have the json file
# containing stock info from YF
SYMBOLS = ['SBIN.NS','ITC.NS','VBL.NS','MRPL.NS','ONGC.NS','HSCL.NS','FCL.NS']

# Path to read stock data from YF
DATA_PATH = 'datasets'

# Dictionary to collect data to create a DF later
data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY' : [],
    'PB': [],
    'ROE' : [],
    'P/S (trail)': [],
    'DPR' : [],
    'DY' : [],
    'CR' : [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
    }

In [15]:
def load_data(json_data):
    data['Symbol'].append(json_data['symbol'])
    data['Name'].append(json_data['longName'])
    data['Industry'].append(json_data['industry'])
    data['Price'].append(json_data['currentPrice'])

    # Could be that some indicators are not available; use NaN if this is the case
    
    if 'forwardEps' in json_data:
        data['EPS (fwd)'].append(json_data['forwardEps'])
    else:
        data['EPS (fwd)'].append(np.nan)
        
    if 'forwardPE' in json_data:
        data['P/E (fwd)'].append(json_data['forwardPE'])
    else:
        data['P/E (fwd)'].append(np.nan)
        
    if 'pegRatio' in json_data:
        data['PEG'].append(json_data['pegRatio'])
    else:
        data['PEG'].append(np.nan)

    if ('freeCashflow' in json_data) and ('marketCap' in json_data):
        fcfy = (json_data['freeCashflow']/json_data['marketCap']) * 100
        data['FCFY'].append(round(fcfy, 2))
    else:
        data['FCFY'].append(np.nan)

    if 'priceToBook' in json_data:
        data['PB'].append(json_data['priceToBook'])
    else:
        data['PB'].append(np.nan)

    if 'returnOnEquity' in json_data:
        data['ROE'].append(json_data['returnOnEquity'])
    else:
        data['ROE'].append(np.nan)
        
    if 'priceToSalesTrailing12Months' in json_data:
        data['P/S (trail)'].append(json_data['priceToSalesTrailing12Months'])
    else:
        data['P/S (trail)'].append(np.nan)

    data['DPR'].append(json_data['payoutRatio'] * 100)

    if 'dividendYield' in json_data:
        data['DY'].append(json_data['dividendYield'])
    else:
        data['DY'].append(0.0)

    if 'beta' in json_data:
        data['Beta'].append(json_data['beta'])
    else:
        data['Beta'].append(np.nan)

    if 'currentRatio' in json_data:
        data['CR'].append(json_data['currentRatio'])
    else:
        data['CR'].append(np.nan)

    if 'fiftyTwoWeekLow' in json_data:
        data['52w Low'].append(json_data['fiftyTwoWeekLow'])
    else:
        data['52w Low'].append(np.nan)
        
    if 'fiftyTwoWeekHigh' in json_data:    
        data['52w High'].append(json_data['fiftyTwoWeekHigh'])
    else:
        data['52w High'].append(np.nan)

In [16]:
for symbol in SYMBOLS:
    # Specify the full path to load JSON data
    file_name = f'{DATA_PATH}/{symbol}.json'    
    try:
        # Open the file in read mode
        with open(file_name, 'r') as file:
            # Use json.load() to parse the JSON data from the file
            load_data(json.load(file))
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [17]:
# Create a DF using the dictionary
df = pd.DataFrame(data)

# Save any stocks with NaN values
df_exceptions = df[df.isna().any(axis=1)]

# Remove any stocks with NaN values
df=df.dropna()

# Reset index after dropping rows with NaN values
df.reset_index(drop=True, inplace=True)

# Add 52 week price range
df['52w Range'] = ((df['Price'] - df['52w Low'])/(df['52w High'] - df['52w Low']))*100

df_exceptions

,Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High
0,SBIN.NS,State Bank of India,Banks - Regional,70.30,8.975818,0.42,NaN,1.428746,0.18680,1.981896,14.79,0.0180,NaN,0.741,631.0,499.35,660.40
2,VBL.NS,Varun Beverages Limited,Beverages - Non-Alcoholic,19.25,65.558440,3.07,NaN,25.267796,NaN,10.517606,11.38,0.0014,NaN,0.542,1262.0,550.00,1380.00
5,HSCL.NS,Himadri Speciality Chemical Limited,Specialty Chemicals,NaN,NaN,NaN,NaN,6.460484,NaN,4.429484,2.92,0.0007,NaN,0.552,373.5,80.20,398.75
6,FCL.NS,Fineotex Chemical Limited,Specialty Chemicals,NaN,NaN,NaN,2.3,9.265433,0.29861,7.041179,8.44,0.0024,2.271,0.976,333.5,203.00,391.90


In [18]:
def make_pretty(styler):
    # Column formatting
    styler.format({'EPS (fwd)': '${:.2f}', 'P/E (fwd)': '{:.2f}', 'PEG': '{:.2f}',
                   'FCFY': '{:.2f}%', 'PB' : '{:.2f}', 'ROE' : '{:.2f}', 'P/S (trail)': '{:.2f}',
                   'DPR': '{:.2f}%', 'DY': '{:.2f}%', 'CR' : '{:.2f}', 'Beta': '{:.2f}', '52w Low': '${:.2f}',
                   'Price': '${:.2f}', '52w High': '${:.2f}', '52w Range': '{:.2f}%'
                  })
    # Set the bar visualization
    styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    styler.background_gradient(subset=['EPS (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['P/E (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['PEG'], cmap='Greens')
    styler.background_gradient(subset=['FCFY'], cmap='Greens')
    styler.background_gradient(subset=['PB'], cmap='Greens')
    styler.background_gradient(subset=['ROE'], cmap='Greens')
    styler.background_gradient(subset=['P/S (trail)'], cmap='Greens')
    styler.background_gradient(subset=['DPR'], cmap='Greens')
    styler.background_gradient(subset=['DY'], cmap='Greens')
    styler.background_gradient(subset=['CR'], cmap='Greens')

    # No index
    styler.hide(axis='index')

    # Tooltips
    styler.set_tooltips(
        ttips, css_class='tt-add',
        props=[
            ('visibility', 'hidden'),
            ('position', 'absolute'),
            ('background-color', 'salmon'),
            ('color', 'black'),
            ('z-index', 1),
            ('padding', '3px 3px'),
            ('margin', '2px')
        ]
    )
    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler


In [19]:
def populate_tt(df, tt_data, col_name):
    stats = df[col_name].describe()
    
    per25 = round(stats.loc['25%'], 2)
    per50 = round(stats.loc['50%'], 2)
    per75 = round(stats.loc['75%'], 2)

    # Get position based on the column name
    pos = df.columns.to_list().index(col_name)
    
    for index, row in df.iterrows():
        pe = row[col_name]
        if pe == stats.loc['min']:
            tt_data[index][pos] = 'Lowest'
        elif pe == stats.loc['max']:
            tt_data[index][pos] = 'Hightest'
        elif pe <= per25:
            tt_data[index][pos] = '25% of companies under {}'.format(per25)
        elif pe <= per50:
            tt_data[index][pos] = '50% of companies under {}'.format(per50)
        elif pe <= per75:
            tt_data[index][pos] = '75% of companies under {}'.format(per75)
        else:
            tt_data[index][pos] = '25% of companies over {}'.format(per75)

In [20]:
# Initialize tool tip data - each column is set to '' for each row
tt_data = [['' for x in range(len(df.columns))] for y in range(len(df))]

# Gather tool tip data for indicators
populate_tt(df, tt_data, 'EPS (fwd)')
populate_tt(df, tt_data, 'P/E (fwd)')
populate_tt(df, tt_data, 'PEG')
populate_tt(df, tt_data, 'FCFY')
populate_tt(df, tt_data, 'PB')
populate_tt(df, tt_data, 'ROE')
populate_tt(df, tt_data, 'P/S (trail)')
populate_tt(df, tt_data, 'DPR')
populate_tt(df, tt_data, 'DY')
populate_tt(df, tt_data, 'CR')

# Create a tool tip DF
ttips = pd.DataFrame(data=tt_data, columns=df.columns, index=df.index)

# Add table caption and styles to DF
df.style.pipe(make_pretty).set_caption('Fundamental Indicators').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])

Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High,52w Range
ITC.NS,ITC Limited,Tobacco,$18.60,25.18,2.10,2.76%,8.45,0.30,8.32,78.70%,0.03%,2.73,0.27,$468.30,$329.10,$499.70,81.59%
MRPL.NS,Mangalore Refinery and Petrochemicals Limited,Oil & Gas Refining & Marketing,$14.90,11.68,0.56,9.38%,2.56,0.38,0.33,0.00%,0.00%,1.07,1.00,$174.00,$49.25,$186.00,91.22%
ONGC.NS,Oil and Natural Gas Corporation Limited,Oil & Gas Integrated,$38.41,6.30,-3.65,14.66%,0.98,0.16,0.51,32.87%,0.04%,1.00,0.72,$241.90,$140.10,$245.45,96.63%


### Key Indicators

- EPS (Earnings Per Share) — portion of a company’s profit that is assigned to each share of its stock
- P/E (Price to Earnings) — relationship between the stock price of a company and its per-share earnings. It helps investors determine if a stock is undervalued or overvalued relative to others in the same sector.
- PEG (Projected Earnings Growth) — calculated by dividing a stock’s P/E by its projected 12-month forward revenue growth rate. In general, a PEG lower than 1 is a good sign, and a PEG higher than 2 indicates that a stock may be overpriced
- FCFY (Free Cash Flow Yield) — a financial solvency ratio that compares the free cash flow per share a company is expected to earn against its market value per share. A lower ratio indicates a less attractive investment opportunity.
- PB (Price to Book) — a ratio of 1 indicates the company’s shares are trading in line with its book value. A P/B higher than 1 suggests the company is trading at a premium to book value, and a P/B lower than 1 indicates a stock that may be undervalued relative to the company’s assets.
- ROE (Return on Equity) — provides a way for investors to evaluate how effectively a company is using its equity to generate profits. A higher ROE indicates a more efficient use of shareholder equity, which can lead to increased demand for shares and higher stock price, as well as increase in company’s profits in the future.
- P/S (Price to Sales) — determines the fair value of a stock by utilizing a company’s market capitalization and revenue. It shows how much the market values the company’s sales, which can be effective in valuing growth stocks that have yet to turn a profit or aren’t performing as expected due to a temporary setback.
- DPR (Dividend Payment Ratio) — a ratio of the total amount of dividends paid out to shareholders relative to the net income of the company.
- DY (Dividend Yield Ratio) — a ratio looks at the amount paid by a company in dividends every year relative to its share price. It is an estimate of the dividend-only return of a stock investment.
- CR (Current Ratio) — measures a company’s ability to pay off its current liabilities (payable within one year) with its current assets, such as cash, accounts receivable, and inventories. The higher the ratio, the better the company’s liquidity position.
- Beta — is a measure of a stock’s volatility in relation to the overall market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock’s beta is less than 1.0.

# To Try

- https://medium.com/@sugath.mudali/a-simple-stock-picking-strategy-with-6-key-financial-metrics-part-1-dab0effb80f8
- github - https://github.com/smudali/stocks-analysis